# TITANIC

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import findspark
findspark.init()

In [2]:
sc= SparkContext(appName= 'Chapter7-Excersice3', master= 'local')
ss= SparkSession(sc)

## Đọc dữ liệu

In [3]:
path= '/Users/vovanthuong/Desktop/9 - Big Data in Machine Learning/Data/Chapter7/titanic.csv'
df= ss.read.csv(path, inferSchema= True, header= True)

In [4]:
df.show(2)

+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|   Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0| PC 17599|71.2833|  C85|       C|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
only showing top 2 rows



In [5]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



## Kiểm tra tổng quát bộ dữ liệu

In [6]:
df.count()

891

In [34]:
for col in df.columns:
    print('________________________{col}________________________'.format(col= col))
    dtype= df.select(col).dtypes[0][1]
    print('dtype:', dtype)
    count= df.select(col).distinct().count()
    print('Number of group:', count)
    cross_table= df.crosstab(col1= col, col2= 'Survived')
    cross_table.withColumn('total', sum(cross_table[col] for col in cross_table.columns[1:])).show()

________________________PassengerId________________________
dtype: int
Number of group: 891
+--------------------+---+---+-----+
|PassengerId_Survived|  0|  1|total|
+--------------------+---+---+-----+
|                 645|  0|  1|    1|
|                  69|  0|  1|    1|
|                 809|  1|  0|    1|
|                 629|  1|  0|    1|
|                 365|  1|  0|    1|
|                 138|  1|  0|    1|
|                 760|  0|  1|    1|
|                 101|  1|  0|    1|
|                 479|  1|  0|    1|
|                 347|  0|  1|    1|
|                 846|  1|  0|    1|
|                 333|  1|  0|    1|
|                 628|  0|  1|    1|
|                 249|  0|  1|    1|
|                 518|  1|  0|    1|
|                 468|  1|  0|    1|
|                 234|  0|  1|    1|
|                 777|  1|  0|    1|
|                 555|  0|  1|    1|
|                 666|  1|  0|    1|
+--------------------+---+---+-----+
only showing top 20 

## Xác định input, output

In [63]:
output_col= 'Survived'
input_cols= df.columns
input_cols.remove(output_col)
input_cols.remove('PassengerId')
input_cols.remove('Name')
input_cols.remove('Ticket')
input_cols.remove('Cabin')

In [50]:
input_cols

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

## Kiểm tra NAN NULL

In [51]:
# Kiểm tra giá trị NaN
from pyspark.sql.functions import count, when, isnan, isnull, col
nan_data= df.select([count(when(isnan(c), c)).alias(c + '_nan') for c in input_cols]).toPandas().T
nan_data

,0
Pclass_nan,0
Sex_nan,0
Age_nan,0
SibSp_nan,0
Parch_nan,0
Fare_nan,0
Embarked_nan,0


In [53]:
# Kiểm tra giá trị Null
null_data= df.select([count(when(isnull(c), c)).alias(c + '_null') for c in input_cols]).toPandas().T
null_data

,0
Pclass_null,0
Sex_null,0
Age_null,177
SibSp_null,0
Parch_null,0
Fare_null,0
Embarked_null,2


In [67]:
df= df.dropna(how= 'any', subset= output_col)
df= df.dropna(how= 'any', subset= input_cols)

Việc loại bỏ hàng dữ liệu NAN hoặc NULL trong trường hợp này nhằm mục đích thực hành bài tập về hồi quy Logistic với Pyspark. Trong điều kiện thực tế việc xóa các dòng dữ liệu chứa NAN/NULL phụ thuộc nhiều vào một số yếu tố cần cân nhắc như: các dữ liệu thiếu (NAN/NULL) là thiếu ngẫu nhiên hay thiếu có hệ thống; số lượng dòng dữ liệu bị thiếu hoặc.  
Đánh giá cá nhân việc sử lý các dữ liệu thiếu: Trong trường hợp thiếu dữ liệu được xác định là ngẫu nhiên, số lượng mẫu nhiều thì nên loại bỏ khỏi dữ liệu train phản ánh chính xác nhất với thực tế và tránh những sai lầm trong việc phỏng đoán dữ liệu thiếu cho mô hình. Nên xây dựng phương án xử lý dữ liệu thiếu cho dữ liệu train trong hầu hết trường hợp bởi lẽ nhu cầu thực tế khi sử dụng mô hình chính là dự đoán trong mọi trường hợp có thể xảy ra.

In [69]:
# Kiểm tra giá trị Null
null_data= df.select([count(when(isnull(c), c)).alias(c + '_null') for c in input_cols]).toPandas().T
null_data

,0
Pclass_null,0
Sex_null,0
Age_null,0
SibSp_null,0
Parch_null,0
Fare_null,0
Embarked_null,0


## Chia tập dữ liệu train và test

In [70]:
train, test= df.randomSplit([0.8, 0.2])

In [71]:
df.count()

712

## Tạo Pipe xử lý dữ liệu

In [101]:
from pyspark.ml.feature import SQLTransformer
select_column= SQLTransformer(statement= 'SELECT {input_cols}, {output_col} FROM __THIS__'.format(input_cols= ','.join(input_cols), output_col= output_col))

In [115]:
from pyspark.ml.feature import StringIndexer
sex_str_indexer= StringIndexer(inputCol= 'Sex', outputCol= 'Sex_idx')
embarked_str_indexer= StringIndexer(inputCol= 'Embarked', outputCol= 'Embarked_idx')

In [117]:
from pyspark.ml.feature import OneHotEncoderEstimator
oh_encoder= OneHotEncoderEstimator(inputCols= ['Embarked_idx'], outputCols= ['Embarked_dummy'])

In [118]:
from pyspark.ml.feature import VectorAssembler
vec_assembler= VectorAssembler(inputCols= ['Pclass', 'Sex_idx', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_dummy'], outputCol= 'features')

In [119]:
from pyspark.ml.feature import MinMaxScaler
mm_scaler= MinMaxScaler(inputCol= 'features', outputCol= 'features_scale')

In [122]:
from pyspark.ml import Pipeline
pipeline_process= Pipeline(stages=[select_column, sex_str_indexer, embarked_str_indexer, oh_encoder, vec_assembler, mm_scaler])

## Tạo mô hình Logistic

In [124]:
process= pipeline_process.fit(train)
train_cleaned= process.transform(train)

In [125]:
from pyspark.ml.classification import LogisticRegression
lgt= LogisticRegression(featuresCol= 'features_scale', labelCol= output_col, predictionCol= output_col + 'predict')
lgt_model= lgt.fit(train_cleaned)

## Đánh giá mô hình

### Đánh giá trên tập train

In [126]:
lgt_evaluate_train= lgt_model.evaluate(train_cleaned)

In [127]:
lgt_evaluate_train.accuracy

0.8125

In [128]:
lgt_evaluate_train.precisionByLabel

[0.8202247191011236, 0.7990196078431373]

In [129]:
lgt_evaluate_train.recallByLabel

[0.8768768768768769, 0.7180616740088106]

In [130]:
lgt_evaluate_train.areaUnderROC

0.8598444259237211

### Đánh giá trên tập test

In [131]:
test_cleaned= process.transform(test)

In [132]:
lgt_evaluate_test= lgt_model.evaluate(test_cleaned)

In [133]:
lgt_evaluate_test.accuracy

0.7828947368421053

In [134]:
lgt_evaluate_test.precisionByLabel

[0.8085106382978723, 0.7413793103448276]

In [135]:
lgt_evaluate_test.recallByLabel

[0.8351648351648352, 0.7049180327868853]

In [136]:
lgt_evaluate_test.areaUnderROC

0.8540803458836244